In [4]:
import pandas as pd
import numpy as np
from sklearn.utils.extmath import randomized_svd
from sklearn.preprocessing import LabelEncoder

#получаем набор данных
products_df = pd.read_csv('products.csv')
transactions_df =  pd.read_csv('transactions.csv')

#вычисляем процент каждого товара от общего числа, купленных каждым юзером
user_product = transactions_df.groupby(['user_id', 'product_id'])['product_id'].count()
user = transactions_df.groupby(['user_id'])['product_id'].count()
g = pd.DataFrame(user_product.div(user, level='user_id')).add_suffix('_count').reset_index()

#присваиваем каждому юзеру и товару порядковые номера
le = LabelEncoder()
g['user_id_num'] = le.fit_transform(g['user_id'])
g['product_id_num'] = le.fit_transform(g['product_id'])


"\ndel transactions_df\n\nrating_matrix = np.empty((max(g_cropped['user_id_num']), \n                          max(g_cropped['product_id_num'])))\n\nrating_matrix.fill(0.0)\nrating_matrix[g_cropped['user_id_num'].values-1, \n              g_cropped['product_id_num'].values-1] = g_cropped['product_id_count'].values\n\nU, Sigma, VT = randomized_svd(rating_matrix, \n                              n_components=30,\n                              n_iter=5,\n                              random_state=10)\n\ng_cropped['product_latent'] = list(map(lambda x: np.array(VT.T[x-1]), \n                                                g_cropped['product_id_num']))\ng_cropped['user_latent'] = list(map(lambda x: np.array(U[x-1]), \n                                                g_cropped['user_id_num']))\n\nusers_emb = pd.DataFrame(g_cropped['user_latent'].to_list())\nusers_emb['user_id'] = g_cropped['user_id']\nusers_emb.drop_duplicates().to_csv('singular_values_users.csv', index = False)\n\nprods_emb =

In [7]:
from keras.models import Model, Sequential
from keras.layers import Embedding, Flatten, Input, concatenate, Dropout, Dense, dot, BatchNormalization, Add
from keras.optimizers import Adam
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import pydot

#строим модель NCF (neural collaborative filtering) paper: https://arxiv.org/pdf/1708.05031.pdf

num_users = len(g.user_id_num.unique())
num_product = len(g.product_id_num.unique())

latent_dim = 10

product_input = Input(shape=[1],name='product-input')
user_input = Input(shape=[1], name='user-input')

product_embedding_mlp = Embedding(num_product + 1, latent_dim, name='product-embedding-mlp')(product_input)
product_vec_mlp = Flatten(name='flatten-movie-mlp')(product_embedding_mlp)

user_embedding_mlp = Embedding(num_users + 1, latent_dim, name='user-embedding-mlp')(user_input)
user_vec_mlp = Flatten(name='flatten-user-mlp')(user_embedding_mlp)

product_embedding_mf = Embedding(num_product + 1, latent_dim, name='product-embedding-mf')(product_input)
product_vec_mf = Flatten(name='flatten-product-mf')(product_embedding_mf)

user_embedding_mf = Embedding(num_users + 1, latent_dim, name='user-embedding-mf')(user_input)
user_vec_mf = Flatten(name='flatten-user-mf')(user_embedding_mf)

concat = concatenate([product_vec_mlp, user_vec_mlp], name='concat')
concat_dropout = Dropout(0.2)(concat)
fc_1 = Dense(100, name='fc-1', activation='relu')(concat_dropout)
fc_1_bn = BatchNormalization(name='batch-norm-1')(fc_1)
fc_1_dropout = Dropout(0.2)(fc_1_bn)
fc_2 = Dense(50, name='fc-2', activation='relu')(fc_1_dropout)
fc_2_bn = BatchNormalization(name='batch-norm-2')(fc_2)
fc_2_dropout = Dropout(0.2)(fc_2_bn)

pred_mlp = Dense(10, name='pred-mlp', activation='relu')(fc_2_dropout)
pred_mf = dot([product_vec_mf, user_vec_mf], normalize = False, axes = -1, name='pred-mf')
combine_mlp_mf = concatenate([pred_mf, pred_mlp], name='combine-mlp-mf')

# Final prediction
result = Dense(1, name='result', activation='relu')(combine_mlp_mf)

model = Model([user_input, product_input], result)
model.compile(optimizer=Adam(lr=0.01), loss='mean_absolute_error')
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
product-input (InputLayer)      (None, 1)            0                                            
__________________________________________________________________________________________________
user-input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
product-embedding-mlp (Embeddin (None, 1, 10)        494660      product-input[0][0]              
__________________________________________________________________________________________________
user-embedding-mlp (Embedding)  (None, 1, 10)        1000010     user-input[0][0]                 
____________________________________________________________________________________________

In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

X_train, X_test = train_test_split(g,test_size=0.4,random_state=42)

history = model.fit([X_train.user_id_num, X_train.product_id_num], X_train.product_id_count, epochs=1)

y_hat = np.round(model.predict([X_test.user_id_num, X_test.product_id_num]), decimals=2)
y_true = X_test.product_id_count
mean_absolute_error(y_true, y_hat)

/home/ilia/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
 601696/5675439 [==>...........................] - ETA: 2:39:27 - loss: 0.0106

In [ ]:
model.save('ncf.h5')

In [3]:
from keras import backend as K
from keras.models import load_model
import pandas as pd

model = load_model('ncf.h5')


ValueError: Unknown layer: Functional

In [64]:
from keras.models import Model, Sequential
from keras.layers import Embedding, Flatten, Input, concatenate, Dropout, Dense, dot, BatchNormalization, Add
from keras.optimizers import Adam
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import pydot

num_users = len(g_cropped.user_id.unique())
num_product = len(g_cropped.product_id.unique())

latent_dim = 10

# Define inputs
product_input = Input(shape=[1],name='product-input')
user_input = Input(shape=[1], name='user-input')

# MLP Embeddings
product_embedding_mlp = Embedding(num_product + 1, latent_dim, name='product-embedding-mlp')(product_input)
product_vec_mlp = Flatten(name='flatten-movie-mlp')(product_embedding_mlp)

user_embedding_mlp = Embedding(num_users + 1, latent_dim, name='user-embedding-mlp')(user_input)
user_vec_mlp = Flatten(name='flatten-user-mlp')(user_embedding_mlp)

# MF Embeddings
product_embedding_mf = Embedding(num_product + 1, latent_dim, name='product-embedding-mf')(product_input)
product_vec_mf = Flatten(name='flatten-product-mf')(product_embedding_mf)

user_embedding_mf = Embedding(num_users + 1, latent_dim, name='user-embedding-mf')(user_input)
user_vec_mf = Flatten(name='flatten-user-mf')(user_embedding_mf)

# MLP layers
concat = concatenate([product_vec_mlp, user_vec_mlp], name='concat')
concat_dropout = Dropout(0.2)(concat)
fc_1 = Dense(100, name='fc-1', activation='relu')(concat_dropout)
fc_1_bn = BatchNormalization(name='batch-norm-1')(fc_1)
fc_1_dropout = Dropout(0.2)(fc_1_bn)
fc_2 = Dense(50, name='fc-2', activation='relu')(fc_1_dropout)
fc_2_bn = BatchNormalization(name='batch-norm-2')(fc_2)
fc_2_dropout = Dropout(0.2)(fc_2_bn)

# Prediction from both layers
pred_mlp = Dense(10, name='pred-mlp', activation='relu')(fc_2_dropout)
pred_mf = dot([product_vec_mf, user_vec_mf], normalize = False, axes = -1, name='pred-mf')
combine_mlp_mf = concatenate([pred_mf, pred_mlp], name='combine-mlp-mf')

# Final prediction
result = Dense(1, name='result', activation='relu')(combine_mlp_mf)

model = Model([user_input, product_input], result)
model.compile(optimizer=Adam(lr=0.01), loss='mean_absolute_error')
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
product-input (InputLayer)      (None, 1)            0                                            
__________________________________________________________________________________________________
user-input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
product-embedding-mlp (Embeddin (None, 1, 10)        186030      product-input[0][0]              
__________________________________________________________________________________________________
user-embedding-mlp (Embedding)  (None, 1, 10)        11000       user-input[0][0]                 
___________________________________________________________________________________________

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

In [65]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

X_train, X_test = train_test_split(g_cropped,test_size=0.10,random_state=42)

history = model.fit([X_train.user_id_num, X_train.product_id_num], X_train.product_id_count, nb_epoch=10)
pd.Series(history.history['loss']).plot(logy=True)
plt.xlabel("Epoch")
plt.ylabel("Train Error")
plt.show()

y_hat = np.round(model.predict([X_test.user_id_num, X_test.product_id_num]), decimals=2)
y_true = X_test.product_id_count
mean_absolute_error(y_true, y_hat)

/home/ilia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  
/home/ilia/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
13952/90000 [===>..........................] - ETA: 1:22 - loss: 0.0160

KeyboardInterrupt: 

,user_id,product_id,product_id_count,user_id_num,product_id_num,product_latent,user_latent
0,1,196,0.169492,0,74,"[0.07366877331002575, -0.125904312002174, -7.5...","[0.03925885227738896, -0.05058399807283374, -2..."
1,1,10258,0.152542,0,3821,"[0.008533846517068464, -0.013478184516794445, ...","[0.03925885227738896, -0.05058399807283374, -2..."
2,1,10326,0.016949,0,3843,"[0.01543997199985665, -0.019374790511282207, -...","[0.03925885227738896, -0.05058399807283374, -2..."
3,1,12427,0.169492,0,4611,"[0.012113173844394086, -0.019309424959549404, ...","[0.03925885227738896, -0.05058399807283374, -2..."
4,1,13032,0.050847,0,4842,"[0.005437302280014471, -0.007218845238512153, ...","[0.03925885227738896, -0.05058399807283374, -2..."
...,...,...,...,...,...,...,...
99995,2297,22035,0.011494,1098,8320,"[0.040165189333049627, 0.006825222197979144, 1...","[0.03925885227738896, -0.05058399807283374, -2..."
99996,2297,22170,0.022989,1098,8373,"[0.001202523938483143, -0.0013002145960813497,...","[0.03925885227738896, -0.05058399807283374, -2..."
99997,2297,23594,0.011494,1098,8890,"[0.0011681389305373596, 7.030058913840115e-05,...","[0.03925885227738896, -0.05058399807283374, -2..."
99998,2297,24830,0.011494,1098,9310,"[0.009150553142294906, 0.0005651100685543683, ...","[0.03925885227738896, -0.05058399807283374, -2..."
